In [2]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample_poly
from scipy.signal import resample_poly, firwin, freqz, lfilter, cheby1, butter
# import matplotlib.pyplot as plt
from scipy import fftpack
import os
from random import randrange
from scipy import signal
from scipy.fftpack import fft, dct
from scipy import stats




In [3]:
path = os.getcwd()
print(path)
sampling_rate = 4000

/home/vips/share/Vu/Snoring-Project/new_code


# Preprocess data

In [ ]:
def cut(frames, n_sample):
    samples = []
    for frame in frames:
        for i in np.arange(0, frame.shape[0], 1):
            s = int(n_sample*(i))
            e = int(n_sample*(i + 1))
            if(e > frame.shape[0]):
                break
            samples.append(frame[s:e])
    return np.array(samples)

def framing(f, spr, t):
    n_sample = t*spr//1000
    for f in os.listdir(path + "/sound/data/non_snoring/"):
        print(f)
        fs, data = wavfile.read(path + "/sound/data/non_snoring/" + f)
        sound = cut(np.array([data]), n_sample)
        np.save(path + "/1s/" + f, np.array(sound))
        print(sound.shape[:])
    return sound.shape[:]
    
def cut_newdata(path, f):
    #read wave file, under wav extension
    fs, data = wavfile.read(path + f + ".wav")
    y = resample_poly(data, 4000, fs).astype(np.int16)
    fs = 4000
    print("sampling rate ", fs, "length ", 1./3600*y.shape[0]/fs)
    start_time = (0*60)*fs
    end_time = (427*60)*fs
    step = 15*60*fs
    while (start_time < end_time):
        s = start_time
        e = start_time + step
        if(e > end_time):
            e = end_time
        temp = y[int(s):int(e)]
        start_time = e
        name = str(int(s/(60*fs))) + "-" + str(int(e/(60*fs))) + ".wav"
        wavfile.write(path + f + "/" + name, fs, temp)

def randomly_cut_30s(path):
    fs, data = wavfile.read(path + "1-13.wav")
    y = resample_poly(data, 4000, fs).astype(np.int16)
    fs = 4000
    print("sampling rate ", fs, "length ", 1./3600*y.shape[0]/fs)
    l = y.shape[0]
    length = fs*30
    for i in range(100):
        start = randrange(l-fs*30)
        sub = y[start:start+length]
        wavfile.write(path + "1-13/" + str(i) + ".wav", fs, sub)

# Feature extraction for ML

In [ ]:
def nor(data):
    mi = np.min(data)*1.0
    ma = np.max(data)*1.0
    data = (2*data - ma - mi)/(ma - mi)
    return data


def stas_feature(data, spr):
    l = data.shape[0]  
    #fft
    yf = fft(data)
    yf = yf[:l//2]
    energy = 1/(l)*np.abs(yf)
    #feature
    r0 = 50*l//spr
    r1 = 250*l//spr
    r2 = 500*l//spr
    r3 = 800*l//spr
     
    mean = np.mean(energy)
    sd = np.std(energy)
    mean1 = np.mean(energy[r0:r1])/mean
    sd1 = np.std(energy[r0:r1])/sd
    mean2 = np.mean(energy[r1:r2])/mean
    sd2 = np.std(energy[r1:r2])/sd
    mean3 = np.mean(energy[r2:r3])/mean
    sd3 = np.std(energy[r2:r3])/sd
    return [mean1, mean2, mean3, sd1, sd2, sd3]

def stas_feature4(data, spr):
    l = data.shape[0]
    #fft
    yf = fft(data)
    yf = yf[:l//2]
    energy = 1/(l)*np.abs(yf)
    energy_scale = energy/scaling_factor
    # energy_scale = np.log(energy + 1)
    energy_scale = energy_scale - np.min(energy_scale)
    #feature
    r0 = 50*l//spr
    r1 = 250*l//spr
    r2 = 500*l//spr
    r3 = 800*l//spr
    # mean = np.mean(energy_scale)
    # sd = np.std(energy_scale)
    mean1 = np.mean(energy_scale[r0:r1])
    sd1 = np.std(energy_scale[r0:r1])
    mean2 = np.mean(energy_scale[r1:r2])
    sd2 = np.std(energy_scale[r1:r2])
    mean3 = np.mean(energy_scale[r2:r3])
    sd3 = np.std(energy_scale[r2:r3])
    return [mean1, mean2, mean3]

def stas_feature6(data, spr):
    l = data.shape[0]  
    #fft
    yf = fft(data)
    yf = yf[:l//2]
    energy = 1/(l)*np.abs(yf)
    energy_scale = energy/scaling_factor
    # energy_scale = np.log(energy + 1)
    energy_scale = energy_scale - np.min(energy_scale)
    #feature

    mean = np.mean(energy_scale)
    sd = np.std(energy_scale)
    skewness = stats.skew(energy_scale)
    kutoris = stats.kurtosis(energy_scale)
    median = np.median(energy_scale)
    
    return [mean, median]

def create_filter_banks(n_filters, spr, l):
    #create filter banks
    filter_banks = np.zeros([n_filters, l//2])
    lower_f = 50
    upper_f = spr//2
    band = np.linspace(2595*np.log(1 + lower_f/700), 2595*np.log(1 + upper_f/700), n_filters + 2)
    band = 700*(np.exp(band/2595) - 1)
    band = np.round(band*1000/(2*spr))
    for i in range(1, n_filters + 1):
        start = int(band[i - 1])
        end = int(band[i + 1])
        mid = int(band[i])
        filter_banks[i - 1][start:mid] = (1.0*np.arange(start, mid) - start)/(mid - start)
        filter_banks[i - 1][mid:end] = (end - 1.0*np.arange(mid, end))/(end - mid)
    return filter_banks

def MFCC(data, filter_banks):
    #fft
    sig = data
    l = sig.shape[0]
    yf = fft(sig)
    yf = yf[:l//2]
    energy = (1/l)*np.abs(yf)
    n_filters = filter_banks.shape[0]
    #filter
    coeff = []
    for i in range(0, n_filters):
        coeff.append(np.log(np.sum(filter_banks[i] * energy)))
    dct_coeff = dct(np.array(coeff))
    return dct_coeff[:n_filters//2]

filter_banks = create_filter_banks(26, sampling_rate, 512)

def feature_extract(data, spr):
    n, l = data.shape[:]
    feature_vectors = []
    hw = np.hamming(l)  
    for i in range(0, n):
        sig = data[i]
        #sig = nor(sig)
        sig = sig - np.mean(sig)
        sig = sig * hw
        #fea = MFCC(sig, filter_banks)
        fea = stas_feature4(sig, spr)
        feature_vectors.append(fea)
    return feature_vectors   

def scaling(path):
    no_snoring = np.load(path)
    data = np.concatenate([no_snoring[0], no_snoring[2], no_snoring[4], no_snoring[6], no_snoring[8], no_snoring[10]])
    l = data.shape[0]
    hw = np.hamming(l)
    data = data * hw
    yf = fft(data)
    yf = yf[:l//2]
    energy = 1/(l)*np.abs(yf)
    return np.mean(energy)


# Machine Learning

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
import os
import time
import pickle
import random

In [ ]:
def train(file_name):
    
    snoring_fea_vecs = np.load(path + "/feature/snoring_" + file_name + ".npy")
    no_snoring_fea_vecs = np.load(path + "/feature/non_snoring_" + file_name  + ".npy")
    n1 = snoring_fea_vecs.shape[0]
    n2 = no_snoring_fea_vecs.shape[0]
    n2 = int(n1)
    print(n1, n2)

    settings = "model" + file_name

    np.random.shuffle(snoring_fea_vecs)
    np.random.shuffle(no_snoring_fea_vecs)

    X = np.concatenate([snoring_fea_vecs, no_snoring_fea_vecs[:n2]], axis=0)
    Y = [0]*n1 + [1]*n2
    t1 = time.time()
    #SVM linear
    clf = svm.SVC(C = 100, kernel='rbf', gamma='scale')
    clf.fit(X,Y)
    pickle.dump(clf, open(path + "/model/" + settings + "/stats_model_SVM_linear.w",'wb')) 

    #train NB
    # clf = GaussianNB()
    # clf.fit(X, Y)
    # pickle.dump(clf, open(path + "/model/" + settings + "/model_NB.w",'wb'))

    #logistic
    # clf = LogisticRegression(random_state=0, solver='lbfgs')
    # clf.fit(X, Y)
    # pickle.dump(clf, open(path + "/model/" + settings + "/model_LR.w",'wb'))

    #tree
    # clf = DecisionTreeClassifier()
    # clf.fit(X, Y)
    # pickle.dump(clf, open(path + "/model/" + settings + "/model_DT.w",'wb'))

    #LDA
    # clf = LinearDiscriminantAnalysis()
    # clf.fit(X, Y)
    # pickle.dump(clf, open(path + "/model/" + settings + "/model_LDA.w",'wb'))

    #GMM
    # clf = GaussianMixture(n_components=2)
    # clf.fit(X)
    # pickle.dump(clf, open(path + "/model/" + settings + "/model_GMM.w",'wb'))

    t2 = time.time()
    print("training time:", t2-t1)

    t1 = time.time()
    r1 = np.sum(1-clf.predict(snoring_fea_vecs[:n1]))/(n1)
    if(r1 < 0.5):
        r1 = 1 - r1
    print("T", r1)
    n2 = no_snoring_fea_vecs.shape[0]
    r2 = np.sum(clf.predict(no_snoring_fea_vecs[:n2]))/(n2)
    if(r2 < 0.5):
        r2 = 1 - r2
    print("F", r2)
    t2 = time.time()
    print("testing time:", (t2-t1))

for i in [1, 4, 5, 7, 8, 9]:
    train(str(i))